# NSIDC ICESat2 Regression tests

### This juypter notebook runs and verifies a series of test requests against NSIDC's ICESat2 data.

Requests are submitted and the retrieved data compared to a set of verified results.

Sample requests include:

- Subset by bounding box
- Subset by temporal range
- Subset by shapefile 


We test against ICESat2 v6 collections:
[ATL03](https://nsidc.org/data/atl03/versions/6), [ATL04](https://nsidc.org/data/atl04/versions/6), [ATL06](https://nsidc.org/data/atl06/versions/6), [ATL07](https://nsidc.org/data/atl07/versions/6), [ATL08](https://nsidc.org/data/atl08/versions/6), [ATL09](https://nsidc.org/data/atl09/versions/6), [ATL10](https://nsidc.org/data/atl10/versions/6), [ATL12](https://nsidc.org/data/atl12/versions/6) and [ATL13](https://nsidc.org/data/atl13/versions/6)


## Prerequisites

The dependencies for this notebook are listed in the
[environment.yaml](https://github.com/nasa/harmony-regression-tests/blob/main/test/nsidc-icesat2/environment.yaml). To test locally, create the conda environment used in the automated regression testing suite:

```sh
conda env create -f ./environment.yaml && conda activate papermill-nsidc-icesat2
```

A `.netrc` file must also be located in the `test` directory of this repository. Additionally the credentials in this .netrc should belong to a user that can access the NSIDC data which may be protected by ACLs in UAT and SIT.


## Set the Harmony environment:

The next cell below sets the `harmony_host_url` to one of the following valid values:

* Production: <https://harmony.earthdata.nasa.gov>
* UAT: <https://harmony.uat.earthdata.nasa.gov>
* SIT: <https://harmony.sit.earthdata.nasa.gov>
* Local: <http://localhost:3000>

By default, the value is set to use Harmony's UAT environment. You can modify the target environment in two ways when using this notebook.

* Run this notebook in a local Jupyter notebook server and simply edit the value of `harmony_host_url` in the cell below to be the desired value for your environment.

* Run the `run_notebooks.sh` script, which uses the papermill library to parameterize and run notebooks. Before running, set the environment variable `HARMONY_HOST_URL` to the desired environment's URL from the list above. This variable will override the default value in the cell below, allowing papermill to inject the correct URL into the notebook at runtime.

In [8]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

### Import required packages

In [3]:
from harmony import BBox, Client, Collection, Dimension, Environment, Request

#### Import shared utility functions:

In [15]:
import sys

sys.path.append('../shared_utils')
from utilities import print_error  # noqa: E402

whoopie


### Set up test information

In [16]:
non_prod_configuration = {
    'subset_bounding_box': {
        'ATL03': {
            'collection_concept_id': 'C1256407609-NSIDC_CUAT',
            'granule_id': 'G1262402516-NSIDC_CUAT',
            'subset': 'subset=lat(40:40.005)&subset=lon(-105.5:-105)',
        },
        'ATL07': {
            'collection_concept_id': 'C1256535488-NSIDC_CUAT',
            'granule_id': 'G1261684946-NSIDC_CUAT',
            'subset': 'subset=lat(80:80.3)&subset=lon(-112:-93)',
        },
        'ATL08': {
            'collection_concept_id': 'C1256432189-NSIDC_CUAT',
            'granule_id': 'G1260745539-NSIDC_CUAT',
            'subset': 'subset=lat(40:40.25)&subset=lon(-105.5:-105)',
        },
    },
    'subset_by_temporal_range': {
        'ATL04': {
            'collection_concept_id': 'C1256535558-NSIDC_CUAT',
            'granule_id': 'G1256952662-NSIDC_CUAT',
            'subset': 'subset=time("2020-04-08T08:00:00.000Z":"2020-04-08T08:05:00.000Z")',
        }
    },
    'subset_by_shapefile': {
        'ATL06': {
            'collection_concept_id': 'C1256358217-NSIDC_CUAT',
            'granule_id': 'G1260779121-NSIDC_CUAT',
            'shape': '-F "shapefile=@Iceland_sliver.geojson;type=application/geo+json"',
        },
    },
}

description|short name|version|root url|collection concept ID|api|api version|collections/all/coverage/rangeset?maxResults=1|granuleId|subset||curl|output file||harmony request url|curl command
----|----|----|----|----|----|----|----|----|----|----|---|----|----|----|----|
subset by bounding box|ATL03|6|https://harmony.uat.earthdata.nasa.gov/|C1256407609-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId|G1262402516-NSIDC_CUAT|subset=lat(40%3A40.25)&subset=lon(-105.5%3A-105)||curl -Lnbj|-o atl03_spatial_test.h5||curl -Lnbj 'https://harmony.uat.earthdata.nasa.gov/C1256407609-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleId&G1262402516-NSIDC_CUAT&subset=lat(40%3A40.25)&subset=lon(-105.5%3A-105)' -o atl03_spatial_test.h5|
subset by temporal range|ATL04|6|https://harmony.uat.earthdata.nasa.gov/|C1256535558-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId|G1256952662-NSIDC_CUAT|subset=time(%222020-04-08T08%3A00%3A00.000Z%22%3A%222020-04-08T08%3A20%3A00.000Z%22)||curl -Lnbj|-o atl04_temporal_test.h5||curl -Lnbj 'https://harmony.uat.earthdata.nasa.gov/C1256535558-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleId&G1256952662-NSIDC_CUAT&subset=time(%222020-04-08T08%3A00%3A00.000Z%22%3A%222020-04-08T08%3A20%3A00.000Z%22)' -o atl04_temporal_test.h5|
subset by shapefile|ATL06|6|https://harmony.uat.earthdata.nasa.gov/|C1256358217-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId|G1260779121-NSIDC_CUAT|||curl -Ln -bj||shapefile=@iceland_simple.zip;type=application/shapefile+zip|curl -Ln -bj 'https://harmony.uat.earthdata.nasa.gov/C1256358217-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleId&G1260779121-NSIDC_CUAT&' |
subset by bounding box|ATL07|6|https://harmony.uat.earthdata.nasa.gov/|C1256535488-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId|G1261684946-NSIDC_CUAT|subset=lat(77%3A82.5)&subset=lon(-112%3A-93)||curl -Lnbj|-o atl07_spatial_test.h5||curl -Lnbj 'https://harmony.uat.earthdata.nasa.gov/C1256535488-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleId&G1261684946-NSIDC_CUAT&subset=lat(77%3A82.5)&subset=lon(-112%3A-93)' -o atl07_spatial_test.h5|
subset by bounding box|ATL08|6|https://harmony.uat.earthdata.nasa.gov/|C1256432189-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId|G1260745539-NSIDC_CUAT|subset=lat(40%3A40.25)&subset=lon(-105.5%3A-105)||curl -Lnbj|-o atl08_spatial_test.h5||curl -Lnbj 'https://harmony.uat.earthdata.nasa.gov/C1256432189-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleId&G1260745539-NSIDC_CUAT&subset=lat(40%3A40.25)&subset=lon(-105.5%3A-105)' -o atl08_spatial_test.h5|
subset by temporal range|ATL08|6|https://harmony.uat.earthdata.nasa.gov/|C1256432189-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId|G1261385533-NSIDC_CUAT|subset=time(%222022-07-31T23%3A01%3A00.000Z%22%3A%222022-07-31T23%3A01%3A30.000Z%22)||curl -Lnbj|-o atl08_temporal_test.h5||curl -Lnbj 'https://harmony.uat.earthdata.nasa.gov/C1256432189-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleId&G1261385533-NSIDC_CUAT&subset=time(%222022-07-31T23%3A01%3A00.000Z%22%3A%222022-07-31T23%3A01%3A30.000Z%22)' -o atl08_temporal_test.h5|
subset by shapefile|ATL09|6|https://harmony.uat.earthdata.nasa.gov/|C1256563776-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId|G1262106425-NSIDC_CUAT|||curl -Ln -bj||shapefile=@Tasmania_simple.geojson;type=application/geo+json|curl -Ln -bj 'https://harmony.uat.earthdata.nasa.gov/C1256563776-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleId&G1262106425-NSIDC_CUAT&' |
subset by shapefile|ATL10|6|https://harmony.uat.earthdata.nasa.gov/|C1256535487-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId||*this test failed because of anti-meridian crossing||curl -Ln -bj||||
subset by bounding box|ATL10|6|https://harmony.uat.earthdata.nasa.gov/|C1256535487-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId|G1261681735-NSIDC_CUAT|||curl -Ln -bj|-o atl10_spatial_test.h5|shapefile=@Ross_sea_positive_lon_only.geojson;type=application/geo+json|curl -Ln -bj 'https://harmony.uat.earthdata.nasa.gov/C1256535487-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleId&G1261681735-NSIDC_CUAT&' -o atl10_spatial_test.h5|
subset by bounding box|ATL12|6|https://harmony.uat.earthdata.nasa.gov/|C1256476536-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId|G1263137424-NSIDC_CUAT|subset=lat(27%3A34)&subset=lon(-79%3A-77)||curl -Ln -bj|-o atl12_spatial_test.h5||curl -Ln -bj 'https://harmony.uat.earthdata.nasa.gov/C1256476536-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleId&G1263137424-NSIDC_CUAT&subset=lat(27%3A34)&subset=lon(-79%3A-77)' -o atl12_spatial_test.h5|
subset by bounding box|ATL13|6|https://harmony.uat.earthdata.nasa.gov/|C1257810199-NSIDC_CUAT|/ogc-api-coverages/|1.0.0/|collections/all/coverage/rangeset?granuleId|G1261681705-NSIDC_CUAT|subset=lat(43%3A45)&subset=lon(-89%3A-75)||curl -Ln -bj|-o atl13_spatial_test.h5||curl -Ln -bj 'https://harmony.uat.earthdata.nasa.gov/C1257810199-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleId&G1261681705-NSIDC_CUAT&subset=lat(43%3A45)&subset=lon(-89%3A-75)' -o atl13_spatial_test.h5|